In [6]:
!pwd

/Users/chengyong/personal/storylabs/backend


In [14]:
!poetry add python-dotenv

Using version ^1.0.1 for python-dotenv

Updating dependencies
Resolving dependencies... (0.7s)

No dependencies to install or update

Writing lock file


In [7]:
#load the original story
with open('app/storybuilder/prompts/original_story.txt', 'r') as file:
    original_story = file.read()

print(original_story)

RAPUNZEL
There were once a man and a woman who had long in vain wished for a child. At length the woman hoped that God was about to grant her desire. These people had a little window at the back of their house from which a splendid garden could be seen, which was full of the most beautiful flowers and herbs. It was, however, surrounded by a high wall, and no one dared to go into it because it belonged to an enchantress, who had great power and was dreaded by all the world. One day the woman was standing by this window and looking down into the garden, when she saw a bed which was planted with the most beautiful rampion (rapunzel), and it looked so fresh and green that she longed for it, she quite pined away, and began to look pale and miserable. Then her husband was alarmed, and asked: ‘What ails you, dear wife?’ ‘Ah,’ she replied, ‘if I can’t eat some of the rampion, which is in the garden behind our house, I shall die.’ The man, who loved her, thought: ‘Sooner than let your wife die,

In [22]:
#load the writer prompt
with open('app/storybuilder/prompts/writer.txt', 'r') as file:
    writer_prompt = file.read()

print(writer_prompt)

You are an expert in writing leveled readers for children.

This is the guide for the reading levels:

{reading_level_guide}

You are given a story. Please rewrite the story as a leveled reader for children at reading level {reading_level}.


In [30]:

with open('app/storybuilder/prompts/reading_level_guide.txt', 'r') as file:
    reading_level_guide = file.read()

print(reading_level_guide)

## Reading Level Guide

### Level 1
- **Description**: Level 1 books are for children who are just beginning to learn about books and reading. At this stage, your child is likely to be learning the alphabet at nursery, and may be able to recognise some simple, common words. Some books are wordless, with pictures telling the story. Wordless books provide lots to talk about, and help your child learn how to hold a book and turn the pages.
- **Approximate age**: 3–4 years old

### Level 1+
- **Description**: Level 1+ books are for children who are just beginning to use simple phonics to read words like ‘sat’ and ’pin’. The books use words that are familiar and predictable, and the pictures help convey the meaning of the story.
- **Approximate age**: 4–5 years old
- **Example**: Taken from *A Dragon Balloon*:
  - "Can Ben grab it? It is fun!"
  - "Get it in the net!"

### Level 2
- **Description**: By this stage, your child will be able to use their knowledge of letters and sounds to work 

In [32]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])



stream = client.chat.completions.create(
    #model="gpt-4o-mini",
    model="gpt-4o-2024-11-20",
    messages=[{"role": "system", "content": f"""{writer_prompt}""".format(reading_level_guide=reading_level_guide, reading_level="2")},
              {"role": "user", "content": f"""This is the original story: {original_story}"""}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o-2024-11-20` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [26]:
#loop through reading levels
for level in range(2, 11):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"""{writer_prompt}""".format(reading_level_guide=reading_level_guide, reading_level=str(level))},
            {"role": "user", "content": f"""This is the original story: {original_story}"""}
        ],
        stream=True,
    )
    
    output_content = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            output_content += chunk.choices[0].delta.content
    
    with open(f"generated_story_level_{level}.txt", "w") as output_file:
        output_file.write(output_content)

In [33]:
messages=[{"role": "system", "content": f"""{writer_prompt}""".format(reading_level_guide=reading_level_guide, reading_level="2")},
              {"role": "user", "content": f"""This is the original story: {original_story}"""}]

In [34]:
print(messages)

[{'role': 'system', 'content': 'You are an expert in writing leveled readers for children.\n\nThis is the guide for the reading levels:\n\n## Reading Level Guide\n\n### Level 1\n- **Description**: Level 1 books are for children who are just beginning to learn about books and reading. At this stage, your child is likely to be learning the alphabet at nursery, and may be able to recognise some simple, common words. Some books are wordless, with pictures telling the story. Wordless books provide lots to talk about, and help your child learn how to hold a book and turn the pages.\n- **Approximate age**: 3–4 years old\n\n### Level 1+\n- **Description**: Level 1+ books are for children who are just beginning to use simple phonics to read words like ‘sat’ and ’pin’. The books use words that are familiar and predictable, and the pictures help convey the meaning of the story.\n- **Approximate age**: 4–5 years old\n- **Example**: Taken from *A Dragon Balloon*:\n  - "Can Ben grab it? It is fun!"\